In [1]:
import os
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

import sistemas
import sei
import functions
import page
from page import *

% reload_ext autoreload
% autoreload 2

In [7]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = webdriver.Firefox()

In [8]:
sei_ = sei.login_sei(browser, usr, pwd)

In [10]:
sei_ = sei.Sei(sei_.driver)

sei_.itera_processos()

ValueError: Verifique o nº de tags de cada linha do         processo: 1. O valor diferente de 6

In [ ]:
sei